# Develop a groundwater model in Flopy

The first step is the delineation of the study area.

We need to import flopy. This library needs to be installed first using the command conda install flopy or another installation routine e.g. with pip install flopy.

In [2]:
import flopy

We need to define a temporary directory. We take the temporary directory of the system to define it. 

In [3]:
# package import
from tempfile import TemporaryDirectory

We define the temporary directory. In the temporary directory we define a name for the workspace "Model01".

In [4]:
temp_dir = TemporaryDirectory()
name = "Model01"
workspace = temp_dir.name

## Defining a model
We now define the model in several steps. First the simulation is defined with the command flopy.mf6.MFSimulation(). This command requires the parameters simulation name and simulation workspace sim_name and sim_ws. The character string is already produced in the variable "workspace".

### Tdis
Second, the temporal discretization is defined using the command flopy.mf6.ModflowTdis(). This command requires 3 parameters: the simulation "sim", which is an object that we have just defined in the previous line, the number of periods "nper", in this case 10. If the variable is nper=1 it is a steady state model, if the variable is > 1, it is a non-steady model. Third, we need the periodical data. We produce 10 arrays with the index 0 to 9 using the loop for _ in range(10). Each time an array of 365 to 1 is produced with the step=1, because of range(start,stop,step). 

### Ims
The command flopy.mf6.ModflowIms is the class used to define the IMS (Iterative Model Solution) package for MODFLOW 6. The IMS package controls how the numerical solver works — including convergence criteria, iteration limits, and solver method parameters. Here we use the default values.

### Gwf
With the command flopy.mf6.ModflowGwf we create an single instance of the groundwater flow model. 

### Dis
With flopy.mf6.ModflowGwfdis(gwf, nlay=3, nrow=4, ncol=5, top=50.0, botm=botm) we define the spatial discretization of the model. It must have as a first parameter the variable "gwf" (which is our model instance), then it has the number of layers and rows and columns and you can specificy a real value for the top (which is always in meters). The default value for botm is 0.

### Fic
With the command flopy.mf6.ModflowGwffic, the model is initialized. A variable strt ([double]) must be specified, which is the  initial (starting) head—that. This is the head at the beginning of the GWF Model simulation. STRT must be specified for all simulations, including steady-state simulations. One value is read for every model cell. For simulations in which the first stress period is steady state, the values used for STRT generally do not affect the simulation (exceptions may occur if cells go dry and (or) rewet). The execution time, however, will be less if STRT includes hydraulic heads that are close to the steady-state solution. A head value lower than the cell bottom can be provided if a cell should start as dry. If it is not specified, it will be inititalized with strt=1.0.

### NPF Node property flow package

The Node Property Flow package is the package used to calculate flow between cells in MODFLOW 6. It is key. We also tell the program to save the specific discharge between cells: With flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True).


### CHD

Finally, we use the CHD package in MODFLOW which is the Time-Variant Specified-Head Package. It is used to assign fixed hydraulic heads to specific cells in a groundwater model, where those heads can change over time.

#### Purpose
Simulates boundary conditions where the hydraulic head is known and enforced. It is useful for modeling:
- Constant head boundaries (e.g., ocean, large lakes, rivers with stable stage)
- Time-varying heads (e.g., tidal effects, seasonal water level changes)
- Transient simulations where head values change between stress periods.

flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 1.0], [(2, 3, 4), 0.0]]). Here the boundary conditions are given for the different layers (as a vector (0,0,0) and (2,3,4) and for different values 1.0 and 0.0.

### Defining where the files are written

Finally, we define, where the files are written. A budget file is defined to contain the water budget

budget_file = f"{name}.bud"

A head file is defined to contain the heads (very important). 

head_file = f"{name}.hds"

The flopy modflow foc package is part of the FloPy library, which is a Python package designed to create, run, and post-process MODFLOW-based models. FloPy supports various MODFLOW versions, including MODFLOW-6, MODFLOW-2005, MODFLOW-NWT, and MODFLOW-USG. It also includes support for other MODFLOW-based models like MODPATH, MT3DMS, MT3D-USGS, and SEAWAT. FloPy requires Python 3.10+. It is available on GitHub and PyPI, with documentation and tutorials provided for users to understand how to use FloPy with MODFLOW models.

flopy.mf6.ModflowGwfoc(
    gwf,
    budget_filerecord=budget_file,
    head_filerecord=head_file,
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)
print("Done creating simulation.")

In [5]:
# set up simulation and basic packages
# Defining the simulation with name and with workspace
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=workspace)
# Defining the temporal discretisation Tdis 
flopy.mf6.ModflowTdis(sim, nper=10, perioddata=[[365.0, 1, 1.0] for _ in range(10)])
flopy.mf6.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)
botm = [30.0, 20.0, 10.0]
flopy.mf6.ModflowGwfdis(gwf, nlay=3, nrow=4, ncol=5, top=50.0, botm=botm)
flopy.mf6.ModflowGwfic(gwf)
flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True)
flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 1.0], [(2, 3, 4), 0.0]])
budget_file = f"{name}.bud"
head_file = f"{name}.hds"
flopy.mf6.ModflowGwfoc(
    gwf,
    budget_filerecord=budget_file,
    head_filerecord=head_file,
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)
print("Done creating simulation.")

Done creating simulation.
